In [2]:
# 1.Daily covid cases

import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime

df=pd.read_excel(r'./DataSource/DailyUpdateForTG.xlsx',skiprows=1,sheet_name=1)
df['城市']=df['指标名称'].str.split(":").str[-1]
df=df.drop(columns=['指标名称','单位','来源','更新时间'])
df.to_hdf('Processed_Data/subway.h5',key='raw')

subway=pd.read_hdf('Processed_Data/subway.h5',key='raw')
subway=pd.melt(subway,id_vars=['城市'],var_name='date',value_name='subway')
subway=subway[subway['subway']>0]

subway['date'] = pd.to_datetime(subway['date']).dt.date
subway=subway.sort_values(by=['城市','date'],ascending=True)
subway['date']=subway['date'].astype('str')


def getdt(d):
    return str(d)[-5:]
subway['dt']=subway['date'].apply(getdt)

subway['subway_last7d_mean']=subway.groupby('城市')['subway'].transform(lambda x: x.rolling(7, 2).mean())
subway['subway_WoW']=subway['subway_last7d_mean']/subway.groupby('城市')['subway_last7d_mean'].transform(lambda x: x.shift(periods=7)) -1

c1=subway['date'].str.contains('2019')
sw_19=subway[c1]
sw_19=sw_19[['城市', 'dt','subway_last7d_mean']]
sw_19=sw_19.rename(columns={'subway_last7d_mean':'subway_19_rolling_mean'})

c2=subway['date'].str.contains('2022|2023')
sw_22=subway[c2]
sw_22=sw_22.merge(sw_19,how='left',on=['城市','dt'])

sw_22['subway_status']=sw_22['subway_last7d_mean']/sw_22['subway_19_rolling_mean']



subway.tail(3)

,城市,date,subway,dt,subway_last7d_mean,subway_WoW
97,长沙,2023-01-05,142.71,01-05,129.644286,0.601705
68,长沙,2023-01-06,156.14,01-06,134.114286,0.496143
39,长沙,2023-01-07,142.87,01-07,135.798571,0.361760


In [3]:
# WoW vs. COVID
%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi
riqi='2023-01-07'

df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
o=sw_22.merge(df_covid,on='城市',how='left')


o=o[o['date']==riqi][o['subway_WoW']<1][['城市','progress_2','subway_WoW']] 

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='本波疫情基本结束'

o=o.sort_values(by='group',ascending=False)
o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-07


In [10]:
# as of 2019 vs. COVID

df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
o=sw_22.merge(df_covid,on='城市',how='left')

o=o[o['subway_status']<1.5][sw_22['date']==riqi]

o=o[['城市','progress_2','subway_status']] 

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='本波疫情基本结束'

o=o.sort_values(by='group',ascending=False)
o.to_clipboard(index=False,header=None)

d=sw_22[sw_22['date']==riqi][['城市','date','subway_status']].reset_index(drop=True).rename(columns={'subway_status':'subway'})
d['date']=pd.to_datetime(d['date'])
d.to_hdf('Processed_Data/city_compare.h5',key='subway')

print(riqi)

2023-01-07


In [11]:
# 2.2 WoW vs. GDP
city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
o=sw_22.merge(city_popu,on='城市',how='left')

o=o[o['date']==riqi][['城市','GDP','subway_WoW']] 

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='本波疫情基本结束'

o=o.sort_values(by='group',ascending=False)
o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-07


In [12]:
# 2.2 as of 2019 vs. GDP
city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
o=sw_22.merge(city_popu,on='城市',how='left')

o=o[o['subway_status']<2][sw_22['date']==riqi]
o=o[['城市','GDP','subway_status']] 

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='本波疫情基本结束'

o=o.sort_values(by='group',ascending=False)
o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-07


In [16]:
#TY copy to wkbk
# d1=pd.read_clipboard()
# d1.to_hdf('Processed_Data/tablehead.h5','subway')

d1=pd.read_hdf('Processed_Data/tablehead.h5','subway')
wkbk=pd.pivot_table(subway[subway['date']>'2023-01-03'],index='城市',columns='date',values='subway',aggfunc='mean').reset_index()
d1.merge(wkbk,how='left',on='城市').drop(columns=['城市']).to_clipboard(index=False)

In [ ]:
# #TG update

# list_tokeep='上海,深圳,广州,成都,西安,北京,苏州,重庆,武汉,郑州,南京'.split(',') #南京
# s1=subway['城市'].isin(list_tokeep)
# df=subway[s1].groupby(by='date').sum().reset_index()
# df2=subway[s1].groupby(by='date').count().reset_index()
# df['subway_7dbefore']=df['subway'].shift(periods=7)
# df['subway_wow']=df['subway']/df['subway'].shift(periods=7) -1

# df['date']=pd.to_datetime(df['date'])

# df['weekday']=df['date'].dt.strftime('%A')

# df['workday']=np.nan
# df['weekend']=np.nan

# c1=df['weekday'].isin(['Saturday','Sunday'])

# df.loc[~c1,'workday']=df['subway']
# df.loc[c1,'weekend']=df['subway']

# df['sum']=np.nan

# df=df[['date','sum','subway_wow','weekend','workday']]

# df=df.sort_values(by='date')
# df.tail(31).to_clipboard(index=False,header=None)
# print(df.tail(10))
# print(df2.tail(5))